In [1]:
from synrxn.data import DataLoader

dl = DataLoader(task="property", source="github", 
                gh_enable=True, resolve_on_init=True)
print(dl.available_names())

['b97xd3', 'cycloadd', 'e2', 'e2sn2', 'lograte', 'phosphatase', 'rad6re', 'rdb7', 'rgd1', 'sn2', 'snar']


In [2]:
from synrxn.split.repeated_kfold import RepeatedKFoldsSplitter
df = dl.load("b97xd3")
splitter = RepeatedKFoldsSplitter(
            n_splits=5,
            n_repeats=5,
            ratio=(8, 1, 1),
            shuffle=True,
            random_state=42,
        )

splitter.split(df, stratify_col=None)

train_df, val_df, test_df = splitter.get_split(repeat=0, fold=0, as_frame=True)